In [38]:
import random
import itertools
import matplotlib.pyplot as plt

S = 0
I = 1
R = 2

# THESE LATER
V = 3 # Vaccinated
Q = 4 # Quarantined


# TODOS: SCHEDULE
# Location indexed by time probably an array
# Indexed by time: Location ID: Int | None

# When simulating you check location and time match

# Locations should be randomly generated and have ID's

# Major based sheduling?

# Should we model parties?

class Location:
    def __init__(self, infection_rate):
        self.infection_rate = infection_rate
        self.bucket = []
        
class Person:
    def __init__(self, state=S):
        self.state = state
        self.recovery_prob = 0.01

def count_state(people, state):
    return sum(person.state == state for person in people)

def gen_locations(num_locations, num_people):
    locs = []

    for i in range(num_locations):
        infection_probability = random.uniform(1e-2, 5e-2)
        loc = Location(infection_probability)
        locs.append(loc)
        
    for j in range(24):
        r = set(range(0, num_people))
        for i in range(num_locations):
            if j < 8 or j > 19:
                locs[i].bucket.append([])
            else:
                samp = set(random.sample([*r], k = num_people // num_locations))
                locs[i].bucket.append(list(samp))
                r = r - samp
    return locs

def gen_people(num_people):
    ppl = []
    for i in range(num_people):
        status = random.choices([I, S], weights=[1,7])[0]
        person = Person(status)
        ppl.append(person)

    return ppl

In [39]:
def people_with_state_by_location(location, people, state, time):
    return filter(lambda t: people[t].state == state, location.bucket[time])

def infection_pairs_by_location(location, people, time):
    infected    = people_with_state_by_location(location, people, I, time)
    susceptible = people_with_state_by_location(location, people, S, time)

    return itertools.product(infected, susceptible)

def find_new_infections_by_location(location, people, time):
    new_infections = []
    for p1, p2 in infection_pairs_by_location(location, people, time):
        if random.random() < location.infection_rate and p2 not in new_infections:
            new_infections.append(p2)
    return new_infections

def simulate2(locations, people, num_days):
    infected = [count_state(people, I)]
    recovered = [count_state(people, R)]
    susceptible = [count_state(people, S)]
    for day in range(num_days):
        for time in range(24):
            # Set Infections
            for location in locations:
                for person_id in find_new_infections_by_location(location, people, time):
                    people[person_id].state = I
            # Handle Recovery
            for person in people:
                if person.state == I and random.random() < person.recovery_prob:
                    person.state = R
        # Get Counts
        infected.append(count_state(people, I))
        recovered.append(count_state(people, R))
        susceptible.append(count_state(people, S))
    return infected, recovered, susceptible

In [ ]:
# In our initial model people got infected really fast but recovered really fast

NUM_LOCS = 100
NUM_PEOPLE = 200_000
NUM_DAYS = 40

locations = gen_locations(NUM_LOCS, NUM_PEOPLE)

ppl = gen_people(num_people=NUM_PEOPLE)
    
i, r, s = simulate2(locations, ppl, NUM_DAYS)

fig, ax = plt.subplots()
ax.plot(i)
ax.plot(r)
ax.plot(s)

ax.legend(["Infected", "Recovered", "Susceptible"])
ax.set_ylabel("People")
ax.set_xlabel("Days")